### **PREPARE THE DATA FOR MACHINE LEARNING ALGORITHMS**

+ We will be writing Functions for this Purpose.

        + Which will help to use the funtionalized code for other Problems.
        
        + Helps to build libraries for Transformations.
        


+ Steps are :--

    + Data Cleaning  :-- SimpleImputer
    
    + Handling Text Attributes.  :-- OrdinalEncoder
    
    + Handling Categorical Attributes.  :-- OneHotEncoder
    
    + Building Custom Transformers.
    
    + Feature Scaling.
    
    + Transformation Pipelines.
    
    + and more Explanations.

In [1]:
## import the libraries
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
housing = pd.read_csv("housing.csv")
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size = 0.2, random_state = 42)
housing["income_cat"] = pd.cut(housing["median_income"],
                              bins = [0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
                              labels = [1,2,3,4,5])
## stratified Splitting
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    #print(train_index), print(len(train_index))
    #print(test_index), print(len(test_index))
    
    strat_train_set = housing.loc[train_index]
    #print(strat_train_set)
    strat_test_set = housing.loc[test_index]
    #print(strat_test_set)

In [9]:
print(housing.shape)
print(strat_train_set.shape)
print(strat_test_set.shape)

(20640, 11)
(16512, 11)
(4128, 11)


In [10]:
## remove the "income_cat" from the stratified sets.
## to get the data as earlier.
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis = 1, inplace = True)
    
    print(strat_train_set.shape)
    print(strat_test_set.shape)

(16512, 10)
(4128, 11)
(16512, 10)
(4128, 10)


In [11]:
print(strat_train_set.shape)
print(strat_test_set.shape)

(16512, 10)
(4128, 10)


Now there is on income_cat in the Sets.

**DATA CLEANING**

+ Starting with Cleaning the Training set.
+ Seperate the Predictors and Labels. Since we don't need to apply these transformations to all Attributes.
+ Data Cleaning is done on Numerical and Category/Text Attributes Seperately.
        + On Numerical :-- Using SimpleImputer.
        + On Category/Text :-- Using OrdinalEncoder or OneHotEncorder

In [12]:
## drop the "mean_house_value" from the training set.
## tehn make a copy of it.
housing_data = strat_train_set.drop("median_house_value", axis = 1)
housing_labels = housing_data.copy()

In [13]:
print(housing_data.shape)
print(housing_labels.shape)

(16512, 9)
(16512, 9)


Most ML models cannot work with Missing Features.

+ Functionalize to take care of them. :-- Missing Values.

+ We had missing values in total_bedrooms attribute Earlier.

+ Three options...
        + Get rid of the Corresponding Districts.
        
        + Get rid of the whole Attribute.
        
        + Set the Values to some values (zero, mean, median etc...)
        
Three Methods..

+ `housing.dropna(subset=["total_bedrooms"])`  :--- Drop Missing Values for Corresponding Districts.
+ `housing.drop("total_bedrooms", axis = 1)`   :--- Drop the Entire Column.
+ `median = housing["total_bedrooms"].median()`
+ `housing["total_bedrooms"].fillna(median, inplace=True)` :--- Imputing with median values.

**HANDLING NUMERICAL ATTRIBUTES - SIMPLE IMPUTER**

Scikit Learn Provides a Handy Class to take Care of Missing Values.

+ Create a Simple Imputer Instance.

+ Specify you are Replacing missing value with Median.

+ Medians can only be Computed on Numerical Attributes.

+ So we need to remove the text Attributes from the Dataset for Now.

+ eg : ocean_proximity - class attribute.

In [16]:
from sklearn.impute import SimpleImputer
## initialize the simple imputer instance 
imputer = SimpleImputer(strategy="median")
## drop the non-numerical attributes for now.
housing_numeric_data = housing_data.drop("ocean_proximity", axis = 1)
## now fit the imputer instance using fit() method
imputer.fit(housing_numeric_data)
## here imputer now simply calculated the median,
## it will calculate the median for all the numerical attributes.
## and will apply imputing to all attributes.
print("Imputer Statistics :--",imputer.statistics_)  ## all values will be saved in the statistics.
print("Median Values which are Calculated :--",housing_numeric_data.median().values)

Imputer Statistics :-- [-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]
Median Values which are Calculated :-- [-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]


In [18]:
## now use the trained imputer to transform the training set by replacing missing values with the learned medians.
X = imputer.transform(housing_numeric_data)
X.shape  ## the result will be a numpy array.

(16512, 8)

In [20]:
## putting the plain Numpy Array to Pandas Dataframe
## name = housing_transformed_simpleimputer
housing_tr_SI = pd.DataFrame(X, columns = housing_numeric_data.columns, index = housing_numeric_data.index)
housing_tr_SI.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347


+ `axis = 0` :-- Verical Axis or Columns.
+ `axis = 1` :-- Horizontal Axis or Rows.
+ `inplace = True` :-- it will modify the underlying data.

**HANDLING CATEGORY-/-TEXT ATTRIBUTES**

+ There is one text Attribute here :-- "ocean_proximity".

+ By Checking, they are Category Values.

+ Most of ML algorithms prefers to work with Numbers.

+ So Converting these Category values to Numbers.

+ from text to numbers.



**ORDINAL ENCODER**

+ Ordianl Encoder used to Convert Categorical Text to Numbers.
+ Converting text values to NUmbers using ORDINAL ENCODING.

+ There is one issue with this Method.
+ ML Algorithms will assume that two nearby values are more similar than two distant values.



In [22]:
housing_data["ocean_proximity"].value_counts()

<1H OCEAN     7276
INLAND        5263
NEAR OCEAN    2124
NEAR BAY      1847
ISLAND           2
Name: ocean_proximity, dtype: int64

In [23]:
housing_data_cat = housing_data[["ocean_proximity"]]

In [25]:
from sklearn.preprocessing import OrdinalEncoder
## initiate the new instance.
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_data_cat)
print("Vaues:--",housing_cat_encoded[:10])
print("Given Categories:--",ordinal_encoder.categories_)

Vaues:-- [[0.]
 [0.]
 [4.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]]
Given Categories:-- [array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype=object)]


**ONEHOT ENCODER**

+ Two avoid the above issue.

+ We use OneHotEncoder method.

+ Create Binary Attribute per Category.

+ Here only one attribute will be having value 1, others will have value 0.

+ The new Attributes are Sometimes Called Dummy Attributes.

+ Sklearn Provides OneHotEncoder to convert Categorical Attributes to one hot vectors.

+ The result will be SciPy Sparse Matrix.

+ We get a matrix with 1000s of Columns.

+ Single 1 per row.

In [27]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_data_cat_1hot = cat_encoder.fit_transform(housing_data_cat)
housing_data_cat_1hot[:10]
print(housing_data_cat_1hot.toarray())

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


+ If a Categorical Attribute Contain large number of Possible Categories.

+ Then one hot encoding will result in a large number of inout features.

+ this may slow down the training and degrade perfomance.

+ If this happens, you may want to replace the categorical Input with useful numerical features related to Category.

+ Here we could replace ocean_proximity feature with the distance to the Ocean.

+ Alternatively, you could replace each category with learnable two dimensional vector called Embedding.

**CUSTOM TRANSFORMERS**

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
    
    
    def __init__(self,add_bedrooms_per_room=True):  ## no *args or **kargs
        self.add_bedrooms_per_room=add_bedrooms_per_room
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        rooms_per_household=X[:,rooms_ix]/X[:,households_ix]
        population_per_household=X[:,population_ix]/X[:,households_ix]
        
        if self.add_bedrooms_per_room:
            bedrooms_per_room=X[:,bedrooms_ix]/X[:,rooms_ix]
            return np.c_[X,rooms_per_household,population_per_household,bedrooms_per_room]
        else:
            return np.c_[X,rooms_per_household,population_per_household]
        
        
    

In [29]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs=attr_adder.transform(housing_data.values)

In [33]:
print(type(housing_extra_attribs))
print(housing_extra_attribs)
print(housing_extra_attribs.shape)

<class 'numpy.ndarray'>
[[-121.89 37.29 38.0 ... '<1H OCEAN' 4.625368731563422 2.094395280235988]
 [-121.93 37.05 14.0 ... '<1H OCEAN' 6.008849557522124 2.7079646017699117]
 [-117.2 32.77 31.0 ... 'NEAR OCEAN' 4.225108225108225 2.0259740259740258]
 ...
 [-116.4 34.09 9.0 ... 'INLAND' 6.34640522875817 2.742483660130719]
 [-118.01 33.82 31.0 ... '<1H OCEAN' 5.50561797752809 3.808988764044944]
 [-122.45 37.77 52.0 ... 'NEAR BAY' 4.843505477308295 1.9859154929577465]]
(16512, 11)


**FEATURE SCALING**

+ Most Important Transformation you need to apply on your data is Feature Scaling.

+ If features have very Different Scale, ML algorithms will not give good Predictions.

+ There are two Methods to do this.

+ Min-Max Scaling and Standardization.

**MIN_MAX SCALING**

+ Values are shifted and rescaled so they end up Ranging from 0 to 1.

+ Scikit learn Provides transformer Called `MinMaxScaler`.

+ It has a feature_range hyperparameter that lets you change the range if you don't want the range 0-1.

**STANDARDIZATION**

+ Standardized value will be having a Zero mean.

+ It Subtract the mean value and divides by the Standard Deviation.

+ standardization = (Value - mean Value) / Standard Deviation.

+ The resulting Distribution will have Unit variance.

+ in Standardization, there will be no range, that will be a problem for Neural Networks.

+ Standardization is less affected by Outliers.

+ Scikit Learn provides transformer Called `StandardScaler`

**TRANSFORMATION PIPELINES**

+ There are many transformation steps we need to do in an Order.

+ Scikit Learn Provides Pipeline class to help with Such sequence of Transfromation.

+ When you call pipeline fit() method, it calls the fit_transform() sequentially on all transformers.

Here is a Small Pipeline for Numerical Attributes.

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

numerical_pipeline = Pipeline([
    ("imputer" , SimpleImputer(strategy="median")),
    ("attribs_adder" , CombinedAttributesAdder()),
    ("std_scaler" , StandardScaler())
])

housing_data_num_transformed = numerical_pipeline.fit_transform(housing_numeric_data)

In [37]:
housing_data_num_transformed.shape

(16512, 11)

Both Numerical and Categorical Attribute transformers in a Single Pipeline.

In [38]:
from sklearn.compose import ColumnTransformer

numerical_attribs = list(housing_numeric_data)
category_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("numeric_values", numerical_pipeline, numerical_attribs),
    ("categorical_values", OneHotEncoder(), category_attribs)
])

In [39]:
housing_prepared = full_pipeline.fit_transform(housing_data)

In [42]:
print(housing_prepared)
print(housing_prepared.shape)
print(type(housing_prepared))

[[-1.15604281  0.77194962  0.74333089 ...  0.          0.
   0.        ]
 [-1.17602483  0.6596948  -1.1653172  ...  0.          0.
   0.        ]
 [ 1.18684903 -1.34218285  0.18664186 ...  0.          0.
   1.        ]
 ...
 [ 1.58648943 -0.72478134 -1.56295222 ...  0.          0.
   0.        ]
 [ 0.78221312 -0.85106801  0.18664186 ...  0.          0.
   0.        ]
 [-1.43579109  0.99645926  1.85670895 ...  0.          1.
   0.        ]]
(16512, 16)
<class 'numpy.ndarray'>


In [43]:
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])